# Eigenmode and EPR analysis Fluxonium

## Sections
1. Prepare the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on single mode. <br>
1. Get qubit freq and anharmonicity. <br>

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

import pyEPR as epr
from qiskit_metal.analyses.quantization import EPRanalysis

In [2]:
from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

### Create the Qubit design

Setup a design of a given dimension. Dimensions will be respected in the design rendering. <br>
Note that the design size extends from the origin into the first quadrant.

In [3]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

design.delete_all_components()

q1 = FluxoniumPocket(design, 'Q1')

gui = MetalGUI(design)


### Finite Element Eigenmode Analysis

#### Setup

Select the analysis you intend to run from the `qiskit_metal.analyses` collection.<br>
Select the design to analyze and the tool to use for any external simulation.

In [4]:
eig_qb = EPRanalysis(design,"hfss")

Review and update the convergence parameters and junction properties by executing following two cells. We exemplify three different methods to update the setup parameters.

In [5]:
eig_qb.sim.setup_update(n_modes = 3,
                        max_delta_f = 0.1,
                        min_freq_ghz = 1,
                        max_passes = 12)
eig_qb.sim.setup.vars.Lj = q1.options.L_j
eig_qb.sim.setup.vars.Ls = q1.options.l_inductance

eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 3,
 'max_delta_f': 0.1,
 'max_passes': 12,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '16.35nH', 'Cj': '0 fF', 'Ls': '200nH'}}

#### Execute simulation and verify convergence and EM field
Analyze a single qubit with shorted terminations. Then observe the frequency convergence plot. If not converging, you might want to increase the min_passes value to force the renderer to increase accuracy.

You can use the method `run()` instead of `sim.run()` in the following cell if you want to run both eigenmode and epr analysis in a single step. If so, make sure to also tweak the setup for the epr analysis. The input parameters are otherwise the same for the two methods.

In [6]:
eig_qb.sim.run(name="FluxoniumEPR", components=['Q1'], open_terminations=[], box_plus_buffer = False)

INFO 03:57PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:57PM [load_ansys_project]: 	Opened Ansys App
INFO 03:57PM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 03:57PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/christianander/Documents/Ansoft/
	Project:   Project12
INFO 03:57PM [connect_design]: No active design found (or error getting active design).
INFO 03:57PM [connect]: 	 Connected to project "Project12". No design detected
INFO 03:57PM [connect_design]: 	Opened active design
	Design:    FluxoniumEPR_hfss [Solution type: Eigenmode]
WARNING 03:57PM [connect_setup]: 	No design setup detected.
WARNING 03:57PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:58PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:58PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:58PM [analyze]: Analyzing setup Setup
04:05PM 38s INFO [get_f_convergence]: Saved convergences to

In [7]:
# eig_qb.sim.renderer.clean_active_design()

In [8]:
# eig_qb.sim.close()

The last variables you pass to the `run()` or `sim.run()` methods, will be stored in the `sim.setup` dictionary under the key `run`. You can recall the information passed by either accessing the dictionary directly, or by using the print handle below.

(optional) Work directly with the convergence numbers

In [9]:
eig_qb.sim.convergence_f

,re(Mode(1)) [g],re(Mode(2)) [g],re(Mode(3)) [g]
Pass [],,,
1,3.947623,11.757697,20.889580
2,5.509198,12.458009,22.437603
3,7.543397,12.878315,23.338145
4,8.561553,13.139591,23.835121
5,8.937411,13.308757,24.129328
6,9.120701,13.426837,24.355987
7,9.197946,13.487978,24.471150
8,9.272473,13.542265,24.574752
9,9.318014,13.583032,24.649891


### EPR Analysis
#### Setup

In [10]:
# Add the geometry of the rectangle  and line of the inductor to the list of junctions
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

eig_qb.add_junction(name_junction ='inductor')
eig_qb.setup.junctions.inductor.Lj_variable = 'Ls'
eig_qb.setup.junctions.inductor.rect = 'JJ_rect_Lj_Q1_inductor'
eig_qb.setup.junctions.inductor.line = 'JJ_Lj_Q1_inductor_'
# eig_qb.setup.junctions.inductor.is_junction = False

# Set up th EPR analysis parameters
eig_qb.setup.fock_trunc = 10
eig_qb.setup.cos_trunc = 6
eig_qb.setup.flux = 0.5

eig_qb.setup.basis = 'HO'

# eig_qb.setup.sweep_variable = 'flux'   #--- This would be nice to implement to get a sweep over flux bias 
eig_qb.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_rect_jj',
   'line': 'JJ_Lj_Q1_rect_jj_'},
  'inductor': {'Lj_variable': 'Ls',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_inductor',
   'line': 'JJ_Lj_Q1_inductor_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 6,
 'fock_trunc': 10,
 'flux': 0.5,
 'basis': 'HO',
 'sweep_variable': 'Lj'}

The name of the `Lj_variable` and `Cj_variable` match with our model. However it is missing the names of the shapes that identify the junction (`rect` and `line`). Look for those in the renderer and find the name. Then let's change the name (See below).

We will now run epr as a single step. On screen you will observe various information in this order:
* stored energy = Electric and magnetic energy stored in the substrate and the system as a whole.
* EPR analysis results for all modes/variations.
* Spectrum analysis.
* Hamiltonian report.

In [11]:
# eig_qb.run_epr()
#### equivalent individual calls
s = eig_qb.setup
eig_qb.epr_start()
eig_qb.get_stored_energy()
eig_qb.run_analysis()
eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
eig_qb.report_hamiltonian(s.sweep_variable)

Design "FluxoniumEPR_hfss" info:
	# eigenmodes    3
	# variations    1
Design "FluxoniumEPR_hfss" info:
	# eigenmodes    3
	# variations    1
Design "FluxoniumEPR_hfss" info:
	# eigenmodes    3
	# variations    1

        energy_elec_all       = 5.03124524792726e-26
        energy_elec_substrate = 4.63876743104179e-26
        EPR of substrate = 92.2%

        energy_mag    = 4.41320963062742e-28
        energy_mag % of energy_elec_all  = 0.9%
        

Variation 0  [1/1]

  Mode 0 at 9.39 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.1%  2.516e-26 2.207e-28

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 89.78%
	jj              0.914007  (+)        0.104052
		Energy fraction (Lj over Lj&Cj)= 41.80%
	inductor        0.0751664  (+)        0.104674
		(U_tot_cap-U_tot_ind)/mean=9.55%
Ca

WARNING 04:07PM [__init__]: <p>Error: <class 'IndexError'></p>
ERROR 04:07PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 199.4% is > 15%.                     
Is the simulation converged? Proceed with caution



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project12\FluxoniumEPR_hfss\2022-03-09 16-06-33.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

[1.635e-08 2.000e-07]
[0]
Using Full Cosine Potential
ϕzpf mode 0 :  0.6561306371400848
ϕzpf mode 1 :  0.00012048352790157151
ϕzpf mode 2 :  0.04237922686097951
Starting the diagonalization
Finished the diagonalization
(3, 1)
Single junctions -- assuming single qubit mode


ERROR 04:07PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 199.4% is > 15%.                     
Is the simulation converged? Proceed with caution


Pm_norm=
modes
0    1.212242e+00
1    4.395899e+07
2    6.406145e+02
dtype: float64

Pm_norm idx =
      jj  inductor
0   True     False
1  False     False
2  False     False
*** P (participation matrix, not normlz.)
             jj      inductor
0  7.561731e-01  6.218648e-02
1  2.126793e-08  1.407729e-09
2  1.449802e-03  1.064958e-04

*** S (sign-bit matrix)
   s_jj  s_inductor
0     1           1
1     1           1
2     1           1
*** P (participation matrix, normalized.)
      0.92
   2.1e-08
    0.0014

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       926 6.25e-05     7.73
  6.25e-05 1.05e-12 2.61e-07
      7.73 2.61e-07   0.0161

*** Chi matrix ND (MHz) 
  8.74e+03-6.73e-06   -0.682
 -6.73e-06 1.14e-10-8.21e-08
    -0.682-8.21e-08  -0.0323

*** Frequencies O1 PT (MHz)
0     8460.366362
1    13647.649349
2    24766.069738
dtype: float64

*** Frequencies ND (MHz)
0       53.063676
1    13647.649359
2    24766.167625
dtype: float64

*** Q

...

###### Numerical diagonalization

Lj,16.35
0,53.06
1,13647.65
2,24766.17


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1         2
Lj                                   
16.35 0  8.74e+03 -6.73e-06 -6.82e-01
      1 -6.73e-06  1.14e-10 -8.21e-08
      2 -6.82e-01 -8.21e-08 -3.23e-02